## CIFAR10 Data Analysis Using the VGG16 Model

This notebook is a continuation of a project to perform a neural network analysis on a graphical dataset. The target dataset is a well-known collection of images known as the CIFAR-10, which is available for download at the following link:

<div class="alert alert-block alert-success">(http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)</div>

The technical report is shown here:
<div class="alert alert-block alert-success">(http://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf)</div>

The task was the train a neural network to be able to identify images, split into 10 classes, based on a training dataset of size 50,000 and a test dataset of 10,000.

The initial analysis was done using first the Multi-Layer Perceptron model. This was followed by a conventional Convolutional Neural Network (CNN) analysis. Owing to the limited CPU resources available, initial runs had to be done on only a subset of the dataset. Once the right modifications had been made to the model, each analysis could be done on the entire dataset. See the 'Results' section below for an account of the GPU-assisted runs and the consequent speed improvement.

## Results

#### Best Accuracy Achieved: 82.7%

In addition to the accuracy, a major performance-related aspect of this mini-project has been runtime. After the CNN runs (with and wihout data augmentation) had taken an inordinate amount of time, the decision was made to switch to using Google colabs, which give the option to run Jupyter Notebooks using GPUs. This step greatly boosted performance and slashed runtimes. Below is a summary of the results for each method:

<BR>


| Model | Full Dataset? | Data Augmented  |     Layers    |  Epochs  | Time Taken | Accuracy |
|-------|---------------|-----------------|---------------|----------|------------|----------|
|MLP    |   Yes         |  No             |      2        |    50    |   10.5 min | 0.486    |
|CNN    |  Yes          |  No             |    1 hidden   |    10    |  3.7 hr    | 0.701    |
|CNN    |  Yes          |  No             |    3 hidden   |    50    |  18.3 hr   | 0.732    |
|CNN    |  Yes          |  Yes            |    3 hidden   |    50    |  23.5 hr   | 0.713    |
|VGG16  |  Yes          |  No             |    16         |    50    |  52.6 min  | 0.798    |
|VGG16  |  Yes          |  Yes            |    16         |    50    |  55.8 min  | 0.827    |
 



In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

import time

Using TensorFlow backend.


In [2]:
# Import Data
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 36s 0us/step


In [3]:
y_train.shape

(50000, 1)

In [0]:
y_test = np.array(y_test)
x_test = np.array(x_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [0]:
# Set image size for VGG Model
img_rows, img_cols = 32,32
# Set number of output classes
num_classes = 10
# Set proportion of entire dataset to be used (e.g. 0.01 = 1%)
prop = 1 # Here we use the full dataset
size1 = int(len(x_train) * prop)
size2 = int(len(x_test) * prop)

x_tr, x_te, y_tr, y_te = x_train[:size1], x_test[:size2], y_train[:size1], y_test[:size2]

if K.image_data_format() == 'channels_first':
    x_tr = x_tr.reshape(x_tr.shape[0], 3, 32, 32)
    x_te = x_te.reshape(x_te.shape[0], 3, 32, 32)
    input_shape = (1, img_rows, img_cols)
else:
    x_tr = x_tr.reshape(x_tr.shape[0], 32, 32, 3)
    x_te = x_te.reshape(x_te.shape[0], 32, 32, 3)
    input_shape = (img_rows, img_cols, 1)

# convert class vectors to binary class matrices
y_tr = keras.utils.to_categorical(y_tr, num_classes)
y_te = keras.utils.to_categorical(y_te, num_classes)

In [6]:
start_time = time.time()

print('About to add hidden layers. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

# Create and instantiate VGG model:
model = Sequential()
weight_decay = 0.0005
input_shape = [32,32,3]

model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

print('About to add hidden layers. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

print('Layer-adding half done. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

print('Layer-adding done, about to compile. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Score trained model.
model.fit(x_tr, y_tr,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(x_te, y_te))
score = model.evaluate(x_te, y_te, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print('*' * 60)
time_taken = '%.1f' % ((time.time() - start_time) / 60)
time_hours = '%.1f' % ((time.time() - start_time) / 3600)
print('Time taken: ' + time_taken + ' minutes ' + '(' + time_hours + ' hours).')

About to add hidden layers. Time taken: 7.232030232747396e-07 minutes
About to add hidden layers. Time taken: 0.023129260540008544 minutes
Layer-adding half done. Time taken: 0.040234665075937905 minutes
Layer-adding done, about to compile. Time taken: 0.053899558385213216 minutes
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 75s 1ms/step - loss: 4.3427 - acc: 0.2481 - val_loss: 4.7186 - val_acc: 0.2119
Epoch 2/50
50000/50000 [==============================] - 63s 1ms/step - loss: 3.5390 - acc: 0.3855 - val_loss: 4.1945 - val_acc: 0.2935
Epoch 3/50
50000/50000 [==============================] - 63s 1ms/step - loss: 2.9782 - acc: 0.4821 - val_loss: 2.9060 - val_acc: 0.4912
Epoch 4/50
50000/50000 [==============================] - 63s 1ms/step - loss: 2.3697 - acc: 0.5956 - val_loss: 3.1476 - val_acc: 0.4484
Epoch 5/50
50000/50000 [==============================] - 63s 1ms/step - loss: 2.0331 - acc: 0.6471 - val_loss: 2.0945 -

In [11]:
start_time = time.time()

print('About to add hidden layers. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

# Create and instantiate VGG model:
model = Sequential()
weight_decay = 0.0005
input_shape = [32,32,3]

model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

print('About to add hidden layers. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

print('Layer-adding half done. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

print('Layer-adding done, about to compile. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Try data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(x_tr)

print('Just fitted data aug. Time taken: ' + str((time.time() - start_time)/60) + ' minutes')

batch_size=256
epochs=50
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_tr, y_tr,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_te, y_te),
                        workers=4)

# Score trained, data-augmented model:

score = model.evaluate(x_te, y_te, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print('*' * 60)
time_taken = '%.1f' % ((time.time() - start_time) / 60)
time_hours = '%.1f' % ((time.time() - start_time) / 3600)
print('Time taken: ' + time_taken + ' minutes ' + '(' + time_hours + ' hours).')

About to add hidden layers. Time taken: 9.179115295410156e-07 minutes
About to add hidden layers. Time taken: 0.002419443925221761 minutes
Layer-adding half done. Time taken: 0.01806903680165609 minutes
Layer-adding done, about to compile. Time taken: 0.03499478499094645 minutes
Just fitted data aug. Time taken: 0.04456682602564494 minutes
Epoch 1/50
196/196 [==============================] - 77s 395ms/step - loss: 4.4199 - acc: 0.2323 - val_loss: 6.2138 - val_acc: 0.2710
Epoch 2/50
196/196 [==============================] - 67s 339ms/step - loss: 3.5939 - acc: 0.3551 - val_loss: 3.3106 - val_acc: 0.4309
Epoch 3/50
196/196 [==============================] - 66s 339ms/step - loss: 2.9852 - acc: 0.4511 - val_loss: 3.3565 - val_acc: 0.3918
Epoch 4/50
196/196 [==============================] - 66s 339ms/step - loss: 2.5742 - acc: 0.5070 - val_loss: 2.6075 - val_acc: 0.4838
Epoch 5/50
196/196 [==============================] - 66s 339ms/step - loss: 2.1891 - acc: 0.5607 - val_loss: 2.1780 -